In [2]:
"""
Complete Motorsports Telemetry Analysis System with Dynamic Dashboard and Interactive Lap Viewer
Barber Motorsports Park - Race Telemetry Visualization and Analysis

Features:
- GPS data loading and cleaning
- Track visualization with speed heatmap
- Sector-based performance analysis
- Ideal racing line calculation
- Comprehensive driver performance report
- Interactive visualizations
- DYNAMIC HTML DASHBOARD GENERATION
- INTERACTIVE LAP-BY-LAP VIEWER

Author: Created for motorsports telemetry analysis
Date: December 2024
"""

import polars as pl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib.colors import Normalize
from sklearn.neighbors import NearestNeighbors
from PIL import Image
import json
from datetime import datetime

# ==================== CONFIGURATION ====================
# Configure these parameters for your analysis
VEHICLE_ID = 'GR86-002-000'
REFERENCE_LAP_NUM = 20
MAX_GAP_THRESHOLD = 30  # meters - for detecting GPS dropouts
MIN_POINTS_PER_LAP = 100
MIN_SPEED_THRESHOLD = 20  # km/h - filter out slow/stopped sections

# File paths
TELEMETRY_CSV = "barber/R1_barber_telemetry_data.csv"
LAP_TIMES_CSV = "barber/R1_barber_lap_time.csv"
TRACK_IMAGE = "barber_track_layout.png"  # Optional reference image
OUTPUT_DIR = "."  # Current directory

# ==================== TRACK CONSTANTS ====================
# Track reference points (lat/lon)
START_FINISH_LAT = 33.5326722 
START_FINISH_LON = -86.6196083
PIT_ENTRY_LAT = 33.531077   
PIT_ENTRY_LON = -86.622592
PIT_EXIT_LAT = 33.531111 
PIT_EXIT_LON = -86.622526

# Distance data (inches - will be converted to meters)
distance_data_in = { 
    'Circuit_Center_Line_Length': 144672,
    'Start_Line_Offset': 0.0,
    'Sector_1_End': 40512, 
    'Sector_2_End': 102732,
    'Sector_3_End': 144672.0,
    'Pit_In_from_SF': 131296.0,
    'Pit_Out_from_SFP': 5352.0,
    'Pit_in_to_Pit_Out': 18794.0
}

INCH_TO_METER = 0.0254
distance_data = {k: v * INCH_TO_METER for k, v in distance_data_in.items()}

# Coordinate system constants
AVG_LAT_DEG = 33.532320
METERS_PER_LAT_DEG = 111320
lat_rad = np.radians(AVG_LAT_DEG)
METERS_PER_LON_DEG = 111320 * np.cos(lat_rad)

# Required telemetry channels
REQUIRED_CHANNELS = [
    'VBOX_Lat_Min', 'VBOX_Long_Minutes', 'Laptrigger_lapdist_dls', 
    'nmot', 'Steering_Angle', 'gear', 'pbrake_r', 'pbrake_f', 
    'accx_can', 'accy_can', 'aps', 'speed'
]

print("="*70)
print("MOTORSPORTS TELEMETRY ANALYSIS SYSTEM")
print("Barber Motorsports Park")
print("="*70)

# ==================== DATA LOADING ====================
def load_telemetry_data(vehicle_id):
    """Load and process telemetry data for specific vehicle"""
    print(f"\n📥 Loading data for {vehicle_id}...")
    
    df_telemetry = (
        pl.scan_csv(TELEMETRY_CSV)
        .filter(pl.col("vehicle_id").str.strip_chars() == vehicle_id)
        .collect()
    )
    
    print(f"   Loaded {len(df_telemetry):,} raw telemetry rows")
    
    # Process telemetry data
    df_track = (
        df_telemetry
        .filter(pl.col("telemetry_name").is_in(REQUIRED_CHANNELS))
        .pivot(
            values="telemetry_value",
            index=["timestamp", "lap", "vehicle_id", "outing"],
            on="telemetry_name",
            aggregate_function="first"
        )
        .rename({
            'VBOX_Lat_Min': 'Latitude', 
            'VBOX_Long_Minutes': 'Longitude',
            'Laptrigger_lapdist_dls': 'LapDist_in',
            'nmot': 'RPM', 
            'Steering_Angle': 'SteeringAngle',
            'pbrake_r': 'BrakeRear', 
            'pbrake_f': 'BrakeFront',
            'accx_can': 'AccX', 
            'accy_can': 'AccY',
            'aps': 'Throttle', 
            'gear': 'Gear',
            'speed': 'Speed_kph'
        })
        .with_columns([
            pl.col('Latitude').cast(pl.Float64, strict=False),
            pl.col('Longitude').cast(pl.Float64, strict=False),
            pl.col('LapDist_in').cast(pl.Float64, strict=False),
            pl.col('RPM').cast(pl.Float64, strict=False),
            pl.col('SteeringAngle').cast(pl.Float64, strict=False),
            pl.col('BrakeRear').cast(pl.Float64, strict=False),
            pl.col('BrakeFront').cast(pl.Float64, strict=False),
            pl.col('AccX').cast(pl.Float64, strict=False),
            pl.col('AccY').cast(pl.Float64, strict=False),
            pl.col('Throttle').cast(pl.Float64, strict=False),
            pl.col('Speed_kph').cast(pl.Float64, strict=False),
            pl.col('Gear').cast(pl.Float64, strict=False)
                .forward_fill()
                .backward_fill()
                .round(0)
                .cast(pl.Int64, strict=False)
        ])
        .sort('timestamp')
    )
    
    print(f"   Processed {len(df_track):,} telemetry points")
    return df_track

# ==================== COORDINATE TRANSFORMATION ====================
def transform_coordinates(df_track):
    """Convert GPS lat/lon to local XY coordinate system"""
    print(f"\n🗺️  Converting coordinates...")
    print(f"   Using {METERS_PER_LON_DEG:.2f} meters per degree longitude")
    
    # Set origin to start/finish line
    min_lon_global = START_FINISH_LON
    min_lat_global = START_FINISH_LAT
    
    # Convert reference points
    start_finish_x = (START_FINISH_LON - min_lon_global) * METERS_PER_LON_DEG
    start_finish_y = (START_FINISH_LAT - min_lat_global) * METERS_PER_LAT_DEG
    pit_entry_x = (PIT_ENTRY_LON - min_lon_global) * METERS_PER_LON_DEG
    pit_entry_y = (PIT_ENTRY_LAT - min_lat_global) * METERS_PER_LAT_DEG
    pit_exit_x = (PIT_EXIT_LON - min_lon_global) * METERS_PER_LON_DEG
    pit_exit_y = (PIT_EXIT_LAT - min_lat_global) * METERS_PER_LAT_DEG
    
    # Transform all telemetry points
    df_track = df_track.with_columns([
        ((pl.col('Longitude') - min_lon_global) * METERS_PER_LON_DEG).alias('X'),
        ((pl.col('Latitude') - min_lat_global) * METERS_PER_LAT_DEG).alias('Y')
    ])
    
    return df_track, (start_finish_x, start_finish_y, pit_entry_x, pit_entry_y, pit_exit_x, pit_exit_y)

# ==================== GPS CLEANING ====================
def clean_gps_data(df, max_gap=30, min_points_per_lap=100):
    """Clean GPS data by filtering laps with too many gaps"""
    print(f"\n🧹 Cleaning GPS data...")
    
    df = df.copy().sort_values(['lap', 'timestamp']).reset_index(drop=True)
    
    # Calculate distances between consecutive points
    df['dx'] = df.groupby('lap')['X'].diff()
    df['dy'] = df.groupby('lap')['Y'].diff()
    df['distance'] = np.sqrt(df['dx']**2 + df['dy']**2)
    
    # Mark large gaps
    df['has_gap'] = df['distance'] > max_gap
    
    # Count gaps per lap
    gaps_per_lap = df.groupby('lap')['has_gap'].sum()
    print(f"   Found gaps in {(gaps_per_lap > 0).sum()} laps")
    
    # Keep only good laps
    points_per_lap = df.groupby('lap').size()
    good_laps = points_per_lap[
        (gaps_per_lap < 10) &
        (points_per_lap > min_points_per_lap)
    ].index
    
    print(f"   Keeping {len(good_laps)} out of {df['lap'].nunique()} laps")
    
    df_clean = df[df['lap'].isin(good_laps)].copy()
    df_clean = df_clean.drop(columns=['dx', 'dy', 'distance', 'has_gap'])
    
    return df_clean

# ==================== SECTOR ASSIGNMENT ====================
def assign_sectors(df, ref_lap_df):
    """Assign sector numbers to each telemetry point"""
    print(f"\n📍 Assigning sectors...")
    
    df = df.copy()
    
    # Calculate cumulative distance for reference lap
    ref_sorted = ref_lap_df.sort_values('timestamp').reset_index(drop=True)
    ref_sorted['segment_dist'] = np.sqrt(
        ref_sorted['X'].diff()**2 + 
        ref_sorted['Y'].diff()**2
    ).fillna(0)
    ref_sorted['cum_dist'] = ref_sorted['segment_dist'].cumsum()
    
    # Find nearest reference point for each telemetry point
    valid_mask = df[['X', 'Y']].notna().all(axis=1)
    valid_indices = df[valid_mask].index
    
    nbrs = NearestNeighbors(n_neighbors=1).fit(ref_sorted[['X', 'Y']].values)
    distances, indices = nbrs.kneighbors(df.loc[valid_indices, ['X', 'Y']].values)
    
    # Assign track distances
    df['track_distance'] = np.nan
    df.loc[valid_indices, 'track_distance'] = ref_sorted.iloc[indices.flatten()]['cum_dist'].values
    
    # Assign sectors based on distance thresholds
    sector_1_end = distance_data['Sector_1_End']
    sector_2_end = distance_data['Sector_2_End']
    
    df['sector'] = np.nan
    df.loc[df['track_distance'] <= sector_1_end, 'sector'] = 1
    df.loc[(df['track_distance'] > sector_1_end) & (df['track_distance'] <= sector_2_end), 'sector'] = 2
    df.loc[df['track_distance'] > sector_2_end, 'sector'] = 3
    
    print(f"   Sector 1: 0m to {sector_1_end:.0f}m")
    print(f"   Sector 2: {sector_1_end:.0f}m to {sector_2_end:.0f}m")
    print(f"   Sector 3: {sector_2_end:.0f}m to end")
    
    return df

# ==================== IDEAL RACING LINE ====================
def calculate_ideal_line(df_with_sectors, segment_length=5):
    """Calculate ideal racing line from best segments across all laps"""
    print(f"\n🏁 Calculating ideal racing line...")
    
    df = df_with_sectors.copy()
    
    # Divide track into segments
    df['segment_id'] = (df['track_distance'] // segment_length).astype(int)
    
    # Find fastest speed for each segment
    best_segments = df.loc[df.groupby('segment_id')['Speed_kph'].idxmax()]
    ideal_line = best_segments.drop_duplicates('segment_id').sort_values('segment_id')
    
    print(f"   Ideal line uses {len(ideal_line)} segments from {df['lap'].nunique()} laps")
    print(f"   Average speed: {ideal_line['Speed_kph'].mean():.1f} km/h")
    
    return ideal_line

# ==================== SECTOR TIME ANALYSIS ====================
def calculate_sector_times(df_with_sectors, min_speed_threshold=20):
    """Calculate sector times using distance/speed to avoid GPS dropout errors"""
    print(f"\n⏱️  Calculating sector times...")
    
    df = df_with_sectors.copy()
    
    # Convert timestamp to datetime
    if not pd.api.types.is_datetime64_any_dtype(df['timestamp']):
        df['timestamp'] = pd.to_datetime(df['timestamp'])
    
    sector_times = []
    
    # Expected sector distances
    sector_distances = {
        1: distance_data['Sector_1_End'],
        2: distance_data['Sector_2_End'] - distance_data['Sector_1_End'],
        3: distance_data['Sector_3_End'] - distance_data['Sector_2_End']
    }
    
    for lap_num in sorted(df['lap'].unique()):
        lap_data = df[df['lap'] == lap_num].sort_values('timestamp').reset_index(drop=True)
        
        for sector in [1, 2, 3]:
            sector_data = lap_data[lap_data['sector'] == sector].copy()
            
            if len(sector_data) < 5:
                continue
            
            # Filter out slow sections (pit lane, etc.)
            sector_data = sector_data[sector_data['Speed_kph'] >= min_speed_threshold].copy()
            
            if len(sector_data) < 5:
                continue
            
            # Calculate distances
            sector_data['dx'] = sector_data['X'].diff()
            sector_data['dy'] = sector_data['Y'].diff()
            sector_data['segment_dist'] = np.sqrt(sector_data['dx']**2 + sector_data['dy']**2)
            
            # Remove unrealistic jumps
            sector_data = sector_data[sector_data['segment_dist'] < 100].copy()
            
            total_distance = sector_data['segment_dist'].sum()
            
            # Validate distance
            expected_dist = sector_distances[sector]
            if total_distance < expected_dist * 0.5 or total_distance > expected_dist * 1.5:
                continue
            
            # Calculate time using distance/speed
            sector_data['speed_ms'] = sector_data['Speed_kph'] / 3.6
            sector_data['segment_time'] = sector_data['segment_dist'] / sector_data['speed_ms']
            
            # Clean time data
            valid_times = sector_data['segment_time'].replace([np.inf, -np.inf], np.nan)
            valid_times = valid_times[valid_times < 10]
            valid_times = valid_times.dropna()
            
            if len(valid_times) < 3:
                continue
            
            sector_time = valid_times.sum()
            
            # Validate sector time (15-60 seconds)
            if sector_time < 15 or sector_time > 60:
                continue
            
            avg_speed = sector_data['Speed_kph'].mean()
            
            sector_times.append({
                'lap': lap_num,
                'sector': sector,
                'time': sector_time,
                'distance': total_distance,
                'avg_speed': avg_speed,
                'max_speed': sector_data['Speed_kph'].max(),
                'min_speed': sector_data['Speed_kph'].min(),
                'points': len(sector_data),
                'coverage': (total_distance / expected_dist) * 100
            })
    
    sector_df = pd.DataFrame(sector_times)
    print(f"   Calculated sector times for {len(sector_df)} sector-lap combinations")
    
    return sector_df

# ==================== COMPLETE LAP ANALYSIS ====================
def analyze_complete_laps(sector_df, vehicle_id):
    """Analyze only laps with all 3 sectors completed"""
    # Count sectors per lap
    sectors_per_lap = sector_df.groupby('lap')['sector'].count()
    complete_laps = sectors_per_lap[sectors_per_lap == 3].index
    
    print(f"\n   Complete laps: {len(complete_laps)} out of {len(sectors_per_lap)}")
    
    # Filter for complete laps
    complete_sector_df = sector_df[sector_df['lap'].isin(complete_laps)].copy()
    
    # Calculate lap times
    lap_times = complete_sector_df.groupby('lap')['time'].sum().sort_values()
    
    # Calculate theoretical best
    theoretical_best = sum(complete_sector_df[complete_sector_df['sector'] == s]['time'].min() for s in [1, 2, 3])
    
    # Print summary
    print("\n" + "="*70)
    print(f"🏁 PERFORMANCE SUMMARY - {vehicle_id}")
    print("="*70)
    
    print(f"\n📊 LAP TIMES:")
    print(f"   Best Lap: Lap {lap_times.idxmin()} - {lap_times.min():.3f}s ({lap_times.min()/60:.2f} min)")
    print(f"   Average: {lap_times.mean():.3f}s ({lap_times.mean()/60:.2f} min)")
    print(f"   Consistency: ±{lap_times.std():.3f}s")
    
    print(f"\n⚡ THEORETICAL BEST LAP: {theoretical_best:.3f}s ({theoretical_best/60:.2f} min)")
    for sector in [1, 2, 3]:
        sector_data = complete_sector_df[complete_sector_df['sector'] == sector]
        best_time = sector_data['time'].min()
        best_lap = sector_data.loc[sector_data['time'].idxmin(), 'lap']
        print(f"   Sector {sector}: {best_time:.3f}s (Lap {best_lap})")
    
    print(f"\n💡 Improvement Potential: {lap_times.min() - theoretical_best:.3f}s")
    
    print(f"\n📈 SECTOR ANALYSIS:")
    for sector in [1, 2, 3]:
        sector_data = complete_sector_df[complete_sector_df['sector'] == sector]
        print(f"\n   Sector {sector}:")
        print(f"      Best: {sector_data['time'].min():.3f}s")
        print(f"      Average: {sector_data['time'].mean():.3f}s ± {sector_data['time'].std():.3f}s")
        print(f"      Avg Speed: {sector_data['avg_speed'].mean():.1f} km/h")
    
    print(f"\n🏆 TOP 5 LAPS:")
    for i, (lap, time) in enumerate(lap_times.head(5).items(), 1):
        lap_sectors = complete_sector_df[complete_sector_df['lap'] == lap]
        s1 = lap_sectors[lap_sectors['sector'] == 1]['time'].values[0]
        s2 = lap_sectors[lap_sectors['sector'] == 2]['time'].values[0]
        s3 = lap_sectors[lap_sectors['sector'] == 3]['time'].values[0]
        print(f"   {i}. Lap {lap:2d}: {time:6.3f}s (S1:{s1:5.2f} S2:{s2:5.2f} S3:{s3:5.2f})")
    
    print("="*70)
    
    return complete_sector_df, lap_times

# ==================== VISUALIZATION FUNCTIONS ====================
def plot_track_comparison(df, df_ref, ideal_line, markers, metric_column='Speed_kph', 
                          image_path=None, max_gap=30):
    """Create side-by-side track comparison with ideal racing line"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 12))
    
    start_finish_x, start_finish_y, pit_entry_x, pit_entry_y, pit_exit_x, pit_exit_y = markers
    
    # LEFT: Reference track layout
    if image_path:
        try:
            img = Image.open(image_path)
            ax1.imshow(img)
            ax1.axis('off')
            ax1.set_title('Official Track Layout', fontsize=16, fontweight='bold')
        except:
            ax1.text(0.5, 0.5, 'Barber Motorsports Park\n17 Turns\n2.38 miles', 
                    ha='center', va='center', fontsize=14)
            ax1.axis('off')
            ax1.set_title('Official Track Layout', fontsize=16, fontweight='bold')
    
    # RIGHT: Telemetry with ideal line
    valid_mask = df[['X', 'Y', metric_column]].notna().all(axis=1)
    df_valid = df[valid_mask].copy().sort_values('timestamp').reset_index(drop=True)
    
    x = df_valid['X'].values
    y = df_valid['Y'].values
    metric_values = df_valid[metric_column].values
    
    # Plot reference lap
    ax2.plot(df_ref['X'].values, df_ref['Y'].values, 
            color='black', alpha=0.5, linewidth=10, 
            label='Track Limits', zorder=1)
    
    # Break into segments at GPS gaps
    distances = np.sqrt(np.diff(x)**2 + np.diff(y)**2)
    gap_indices = np.where(distances > max_gap)[0]
    
    segment_starts = [0] + (gap_indices + 1).tolist()
    segment_ends = gap_indices.tolist() + [len(x) - 1]
    
    norm = Normalize(metric_values.min(), metric_values.max())
    
    # Plot each continuous segment
    for start, end in zip(segment_starts, segment_ends):
        if end > start:
            x_seg = x[start:end+1]
            y_seg = y[start:end+1]
            metric_seg = metric_values[start:end+1]
            
            points = np.array([x_seg, y_seg]).T.reshape(-1, 1, 2)
            segments = np.concatenate([points[:-1], points[1:]], axis=1)
            
            lc = LineCollection(segments, cmap='plasma', norm=norm, linewidth=4, zorder=5)
            lc.set_array(metric_seg[:-1])
            ax2.add_collection(lc)
    
    # Plot ideal line
    ax2.plot(ideal_line['X'], ideal_line['Y'],
            color='red', linewidth=3, alpha=0.7, linestyle='--',
            label='Ideal Racing Line', zorder=6)
    
    # Add markers
    ax2.scatter(start_finish_x, start_finish_y, 
              color='lime', marker='*', s=500, edgecolor='black', 
              linewidth=3, label='START/FINISH', zorder=15)
    ax2.scatter(pit_entry_x, pit_entry_y, 
              color='orange', marker='s', s=300, edgecolor='black', 
              linewidth=2, label='Pit Entry', zorder=10)
    ax2.scatter(pit_exit_x, pit_exit_y,
              color='red', marker='s', s=300, edgecolor='black', 
              linewidth=2, label='Pit Exit', zorder=10)
    
    # Formatting
    ax2.set_xlim(x.min() - 50, x.max() + 50)
    ax2.set_ylim(y.min() - 50, y.max() + 50)
    ax2.set_aspect('equal', adjustable='box')
    ax2.set_facecolor('#e8e8e8')
    ax2.grid(True, alpha=0.3)
    
    # Colorbar
    dummy_lc = LineCollection([[(0, 0), (1, 1)]], cmap='plasma', norm=norm)
    dummy_lc.set_array(metric_values)
    plt.colorbar(dummy_lc, ax=ax2, label=metric_column)
    
    ax2.set_title(f'Telemetry Data - {VEHICLE_ID}', fontsize=16, fontweight='bold')
    ax2.set_xlabel('Local X Coordinate (meters)', fontsize=12)
    ax2.set_ylabel('Local Y Coordinate (meters)', fontsize=12)
    ax2.legend(loc='upper left', fontsize=11)
    
    plt.tight_layout()
    return fig

def plot_performance_analysis(complete_sectors, lap_times, vehicle_id):
    """Create comprehensive performance analysis charts"""
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    theoretical_best = sum(complete_sectors[complete_sectors['sector'] == s]['time'].min() for s in [1, 2, 3])
    
    # Plot 1: Sector times by lap
    ax = axes[0, 0]
    for sector in [1, 2, 3]:
        sector_data = complete_sectors[complete_sectors['sector'] == sector].sort_values('lap')
        ax.plot(sector_data['lap'], sector_data['time'], 'o-', 
               label=f'Sector {sector}', linewidth=2, markersize=5)
        
        # Mark best
        best_idx = sector_data['time'].idxmin()
        best_lap = sector_data.loc[best_idx, 'lap']
        best_time = sector_data.loc[best_idx, 'time']
        ax.scatter([best_lap], [best_time], s=200, zorder=10, 
                  edgecolor='black', linewidth=2)
    
    ax.set_xlabel('Lap Number', fontsize=12)
    ax.set_ylabel('Sector Time (seconds)', fontsize=12)
    ax.set_title('Sector Times Across Laps', fontsize=14, fontweight='bold')
    ax.legend(fontsize=11)
    ax.grid(True, alpha=0.3)
    
    # Plot 2: Lap times
    ax = axes[0, 1]
    lap_times_sorted = lap_times.sort_index()
    ax.plot(lap_times_sorted.index, lap_times_sorted.values, 
           'o-', color='purple', linewidth=2, markersize=6)
    ax.axhline(theoretical_best, color='red', linestyle='--', linewidth=2, 
              label=f'Theoretical Best: {theoretical_best:.2f}s')
    ax.axhline(lap_times.mean(), color='orange', linestyle='--', linewidth=2,
              label=f'Average: {lap_times.mean():.2f}s')
    
    # Mark best lap
    best_lap = lap_times.idxmin()
    best_time = lap_times.min()
    ax.scatter([best_lap], [best_time], s=300, color='gold', 
              zorder=10, edgecolor='black', linewidth=3, marker='*')
    
    ax.set_xlabel('Lap Number', fontsize=12)
    ax.set_ylabel('Lap Time (seconds)', fontsize=12)
    ax.set_title('Lap Time Progression', fontsize=14, fontweight='bold')
    ax.legend(fontsize=10)
    ax.grid(True, alpha=0.3)
    
    # Plot 3: Sector consistency
    ax = axes[1, 0]
    sector_times_list = []
    sector_data_list = []
    for sector in [1, 2, 3]:
        sector_data = complete_sectors[complete_sectors['sector'] == sector]
        sector_times_list.append(sector_data['time'].values)
        sector_data_list.append(sector_data)
    
    bp = ax.boxplot(sector_times_list, tick_labels=[f'Sector {s}' for s in [1, 2, 3]], 
                   patch_artist=True)
    colors = ['lightblue', 'lightgreen', 'lightcoral']
    for patch, color in zip(bp['boxes'], colors):
        patch.set_facecolor(color)
    
    # Label outliers with lap numbers - improved non-overlapping algorithm
    for i, (sector, sector_data) in enumerate(zip([1, 2, 3], sector_data_list), 1):
        # Calculate outliers using IQR method (same as boxplot)
        times = sector_data['time'].values
        q1 = np.percentile(times, 25)
        q3 = np.percentile(times, 75)
        iqr = q3 - q1
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr
        
        # Find outliers and sort by time
        outliers = sector_data[(sector_data['time'] < lower_bound) | (sector_data['time'] > upper_bound)]
        outliers = outliers.sort_values('time').reset_index(drop=True)
        
        if len(outliers) == 0:
            continue
        
        # Create list of (lap_number, actual_time, label_y_position)
        outlier_info = []
        min_spacing = 1.0  # Minimum vertical spacing between labels
        
        for idx, row in outliers.iterrows():
            outlier_info.append({
                'lap': int(row['lap']),
                'actual_y': row['time'],
                'label_y': row['time']  # Start with actual position
            })
        
        # Iteratively adjust positions to eliminate overlaps
        max_iterations = 20
        for iteration in range(max_iterations):
            overlaps_fixed = True
            for j in range(len(outlier_info) - 1):
                current = outlier_info[j]
                next_label = outlier_info[j + 1]
                
                # Check if labels overlap
                if next_label['label_y'] - current['label_y'] < min_spacing:
                    overlaps_fixed = False
                    # Push them apart
                    midpoint = (current['label_y'] + next_label['label_y']) / 2
                    current['label_y'] = midpoint - min_spacing / 2
                    next_label['label_y'] = midpoint + min_spacing / 2
            
            if overlaps_fixed:
                break
        
        # Draw labels and connection lines
        for info in outlier_info:
            lap_num = info['lap']
            actual_y = info['actual_y']
            label_y = info['label_y']
            
            # Draw connection line if label was moved significantly
            if abs(label_y - actual_y) > 0.15:
                ax.plot([i, i + 0.12], [actual_y, label_y], 'k-', linewidth=0.5, alpha=0.4)
            
            # Add label with white background
            ax.text(i + 0.15, label_y, f'{lap_num}', fontsize=7.5, ha='left', va='center', 
                   color='darkred', fontweight='bold',
                   bbox=dict(boxstyle='round,pad=0.25', facecolor='white', 
                            edgecolor='darkred', linewidth=0.8, alpha=0.95))
    
    ax.set_ylabel('Time (seconds)', fontsize=12)
    ax.set_title('Sector Time Consistency', fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3, axis='y')
    
    # Plot 4: Speed by sector
    ax = axes[1, 1]
    sectors = [1, 2, 3]
    avg_speeds = [complete_sectors[complete_sectors['sector'] == s]['avg_speed'].mean() 
                 for s in sectors]
    max_speeds = [complete_sectors[complete_sectors['sector'] == s]['max_speed'].max() 
                 for s in sectors]
    
    x = np.arange(len(sectors))
    width = 0.35
    
    ax.bar(x - width/2, avg_speeds, width, label='Average Speed', color='steelblue')
    ax.bar(x + width/2, max_speeds, width, label='Max Speed', color='coral')
    
    ax.set_xlabel('Sector', fontsize=12)
    ax.set_ylabel('Speed (km/h)', fontsize=12)
    ax.set_title('Speed Analysis by Sector', fontsize=14, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels([f'Sector {s}' for s in sectors])
    ax.legend(fontsize=11)
    ax.grid(True, alpha=0.3, axis='y')
    
    plt.suptitle(f'Performance Analysis - {vehicle_id} at Barber Motorsports Park', 
                fontsize=16, fontweight='bold')
    plt.tight_layout()
    return fig

# ==================== DYNAMIC HTML DASHBOARD GENERATION ====================
def generate_unified_dashboard(complete_sectors, lap_times, vehicle_id, df_with_sectors):
    """
    Generate unified HTML dashboard with all features
    """
    
    # Import distance_data from global scope
    from __main__ import distance_data
    
    # Get list of complete laps
    sectors_per_lap = complete_sectors.groupby('lap')['sector'].count()
    complete_laps = sorted(sectors_per_lap[sectors_per_lap == 3].index.tolist())
    
    # Calculate statistics
    theoretical_best = sum(complete_sectors[complete_sectors['sector'] == s]['time'].min() for s in [1, 2, 3])
    best_lap_num = int(lap_times.idxmin())
    best_lap_time = lap_times.min()
    avg_lap_time = lap_times.mean()
    improvement = best_lap_time - theoretical_best
    complete_laps_count = len(lap_times)
    
    # Calculate average speed and track length
    avg_speed = df_with_sectors['Speed_kph'].mean()
    track_length = distance_data['Circuit_Center_Line_Length'] / 1000
    
    # Get sector stats
    sector_stats = []
    for sector in [1, 2, 3]:
        sector_data = complete_sectors[complete_sectors['sector'] == sector]
        best_time = sector_data['time'].min()
        best_lap = int(sector_data.loc[sector_data['time'].idxmin(), 'lap'])
        avg_time = sector_data['time'].mean()
        std_time = sector_data['time'].std()
        avg_speed_sector = sector_data['avg_speed'].mean()
        
        sector_stats.append({
            'sector': sector,
            'best_time': best_time,
            'best_lap': best_lap,
            'avg_time': avg_time,
            'std_time': std_time,
            'avg_speed': avg_speed_sector
        })
    
    # Get top 5 laps
    top_laps = []
    for i, (lap, time) in enumerate(lap_times.head(5).items(), 1):
        lap_sectors = complete_sectors[complete_sectors['lap'] == lap]
        s1 = lap_sectors[lap_sectors['sector'] == 1]['time'].values[0]
        s2 = lap_sectors[lap_sectors['sector'] == 2]['time'].values[0]
        s3 = lap_sectors[lap_sectors['sector'] == 3]['time'].values[0]
        
        top_laps.append({
            'rank': i,
            'lap': int(lap),
            'time': time,
            's1': s1,
            's2': s2,
            's3': s3,
            'is_best': i == 1
        })
    
    # Sector boundaries
    sector_boundaries = {
        1: (0, distance_data['Sector_1_End']),
        2: (distance_data['Sector_1_End'], distance_data['Sector_2_End']),
        3: (distance_data['Sector_2_End'], distance_data['Circuit_Center_Line_Length'])
    }
    
    total_points = len(df_with_sectors)
    
    # Get best sector information
    best_sectors = {}
    for sector in [1, 2, 3]:
        sector_data = complete_sectors[complete_sectors['sector'] == sector]
        best_time = sector_data['time'].min()
        best_lap = int(sector_data.loc[sector_data['time'].idxmin(), 'lap'])
        best_sectors[sector] = {'lap': best_lap, 'time': best_time}
    
    # Prepare JSON data for lap viewer
    lap_data_json = {}
    for lap in complete_laps:
        lap_telemetry = df_with_sectors[df_with_sectors['lap'] == lap].copy()
        
        # Get sector times
        lap_sector_times = {}
        for sector in [1, 2, 3]:
            sector_times = complete_sectors[(complete_sectors['lap'] == lap) & 
                                          (complete_sectors['sector'] == sector)]
            if len(sector_times) > 0:
                lap_sector_times[sector] = float(sector_times['time'].values[0])
        
        # Use ALL data points (no sampling)
        lap_data_json[int(lap)] = {
            'x': lap_telemetry['X'].tolist(),
            'y': lap_telemetry['Y'].tolist(),
            'speed': lap_telemetry['Speed_kph'].tolist(),
            'gear': lap_telemetry['Gear'].fillna(0).astype(int).tolist(),
            'sector': lap_telemetry['sector'].fillna(0).astype(int).tolist(),
            'sector_times': lap_sector_times,
            'total_time': sum(lap_sector_times.values())
        }
    
    # Convert to JSON
    lap_data_str = json.dumps(lap_data_json)
    best_sectors_str = json.dumps(best_sectors)
    
    most_inconsistent = max(sector_stats, key=lambda x: x['std_time'])
    potential_gain = most_inconsistent['avg_time'] - most_inconsistent['best_time']
    performance_pct = (theoretical_best / best_lap_time) * 100
    
    # Calculate filtered laps
    all_laps_in_range = set(range(1, max(complete_laps) + 1))
    complete_laps_set = set(complete_laps)
    filtered_laps = sorted(all_laps_in_range - complete_laps_set)
    
    # Start building HTML
    html = '''<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8">
<meta name="viewport" content="width=device-width, initial-scale=1.0">
<title>Barber Motorsports Park - ''' + vehicle_id + ''' Telemetry Analysis</title>
<script crossorigin src="https://unpkg.com/react@18/umd/react.production.min.js"></script>
<script crossorigin src="https://unpkg.com/react-dom@18/umd/react-dom.production.min.js"></script>
<script src="https://unpkg.com/@babel/standalone/babel.min.js"></script>
<style>
@import url('https://fonts.googleapis.com/css2?family=Orbitron:wght@400;700;900&family=Rajdhani:wght@300;400;600;700&display=swap');

:root {
    --bg-dark: #0a0e14;
    --bg-card: #13171f;
    --accent-red: #ff3366;
    --accent-cyan: #00d9ff;
    --accent-yellow: #ffd700;
    --text-primary: #e8e8e8;
    --text-secondary: #9ca3af;
    --border-color: #2d3748;
}

* { margin: 0; padding: 0; box-sizing: border-box; }

body {
    font-family: 'Rajdhani', sans-serif;
    background: var(--bg-dark);
    color: var(--text-primary);
}

.header {
    background: linear-gradient(135deg, var(--bg-card) 0%, rgba(255, 51, 102, 0.1) 100%);
    border-bottom: 2px solid var(--accent-red);
    padding: 2rem;
}

.track-name {
    font-family: 'Orbitron', sans-serif;
    font-size: 3rem;
    font-weight: 900;
    background: linear-gradient(135deg, var(--accent-red), var(--accent-cyan));
    -webkit-background-clip: text;
    -webkit-text-fill-color: transparent;
    text-transform: uppercase;
}

.vehicle-id {
    font-family: 'Orbitron', sans-serif;
    font-size: 1.5rem;
    font-weight: 700;
    color: var(--accent-yellow);
    margin-top: 0.5rem;
}

.stats-bar {
    display: grid;
    grid-template-columns: repeat(auto-fit, minmax(180px, 1fr));
    gap: 1rem;
    margin-top: 1.5rem;
}

.stat-card {
    background: rgba(255, 255, 255, 0.05);
    border: 1px solid var(--border-color);
    border-left: 3px solid var(--accent-cyan);
    padding: 1rem;
    position: relative;
    overflow: hidden;
    transition: all 0.3s ease;
}

.stat-card::before {
    content: '';
    position: absolute;
    top: 0;
    left: 0;
    width: 100%;
    height: 100%;
    background: linear-gradient(135deg, transparent, rgba(0, 217, 255, 0.1));
    opacity: 0;
    transition: opacity 0.3s ease;
}

.stat-card:hover::before { opacity: 1; }
.stat-card:hover {
    transform: translateY(-2px);
    border-left-color: var(--accent-yellow);
}

.stat-label {
    font-size: 0.75rem;
    color: var(--text-secondary);
    text-transform: uppercase;
    letter-spacing: 0.1em;
    margin-bottom: 0.25rem;
    position: relative;
    z-index: 1;
}

.stat-value {
    font-family: 'Orbitron', sans-serif;
    font-size: 1.75rem;
    font-weight: 700;
    color: var(--text-primary);
    position: relative;
    z-index: 1;
}

.stat-unit {
    font-size: 1rem;
    color: var(--text-secondary);
    margin-left: 0.25rem;
}

.container {
    max-width: 1600px;
    margin: 0 auto;
    padding: 2rem;
}

.tabs {
    display: flex;
    gap: 1rem;
    margin-bottom: 2rem;
    border-bottom: 2px solid var(--border-color);
}

.tab {
    font-family: 'Orbitron', sans-serif;
    background: none;
    border: none;
    color: var(--text-secondary);
    padding: 1rem 2rem;
    font-size: 1rem;
    font-weight: 700;
    text-transform: uppercase;
    cursor: pointer;
    transition: color 0.3s;
}

.tab.active { color: var(--accent-red); border-bottom: 3px solid var(--accent-red); }
.tab:hover { color: var(--accent-cyan); }

.tab-content { display: none; }
.tab-content.active { display: block; }

table {
    width: 100%;
    border-collapse: collapse;
}

table th {
    text-transform: uppercase;
    font-size: 0.85rem;
    letter-spacing: 0.1em;
    padding: 1rem;
    text-align: left;
}

table td {
    padding: 1rem;
    border-bottom: 1px solid var(--border-color);
}

table tr:hover {
    background: rgba(0, 217, 255, 0.05);
}

#lap-viewer-container {
    display: flex;
    width: 100%;
    height: calc(100vh - 300px);
    min-height: 600px;
}

.canvas-container {
    flex: 1;
    padding: 20px;
}

#lap-canvas {
    width: 100%;
    height: 100%;
    border: 1px solid var(--border-color);
    border-radius: 8px;
    background: var(--bg-card);
}

.viewer-panel {
    width: 400px;
    background: var(--bg-card);
    border-left: 2px solid var(--border-color);
    padding: 30px;
    display: flex;
    flex-direction: column;
    gap: 20px;
    overflow-y: auto;
}

.mode-button {
    flex: 1;
    padding: 12px;
    background: rgba(255, 255, 255, 0.05);
    border: 1px solid var(--border-color);
    border-radius: 4px;
    color: #e8e8e8;
    font-family: 'Rajdhani', sans-serif;
    font-weight: 600;
    cursor: pointer;
    transition: all 0.3s;
}

.mode-button.active {
    background: var(--accent-red);
    border-color: var(--accent-red);
}

.nav-button {
    flex: 1;
    padding: 15px;
    background: var(--accent-red);
    border: none;
    border-radius: 4px;
    color: white;
    font-family: 'Orbitron', sans-serif;
    font-weight: 700;
    cursor: pointer;
}

.nav-button:disabled {
    background: rgba(255, 255, 255, 0.02);
    color: #4a5568;
    cursor: not-allowed;
}
</style>
</head>
<body>

<header class="header">
<h1 class="track-name">Barber Motorsports Park</h1>
<div class="vehicle-id">''' + vehicle_id + '''</div>
'''
    
    # Add data quality note if laps were filtered
    if filtered_laps:
        html += '''
<div style="margin-top: 1rem; padding: 1rem; background: rgba(255, 215, 0, 0.1); border-left: 3px solid var(--accent-yellow); border-radius: 4px; max-width: 1200px;">
<div style="font-size: 0.9rem; font-weight: 600; color: var(--accent-yellow); margin-bottom: 0.5rem;">📋 Data Quality Note</div>
<div style="font-size: 0.85rem; color: var(--text-secondary); line-height: 1.6;">
<strong>Analyzed ''' + str(complete_laps_count) + ''' complete laps</strong> (laps with all 3 sector times recorded). 
<strong>Filtered: Lap''' + ('s' if len(filtered_laps) > 1 else '') + ' ' + ', '.join(map(str, filtered_laps)) + '''</strong> 
(incomplete sector data). <br>
<strong>⚠️ Important:</strong> Charts may show all lap numbers on the X-axis for continuity, but data points and 
<strong>ALL statistics (averages, best times, consistency)</strong> are calculated ONLY from the 
''' + str(complete_laps_count) + ''' complete laps. Filtered laps appear as gaps/missing points in the charts.
</div>
</div>
'''
    
    # Stats Bar
    html += '''
<div class="stats-bar">
<div class="stat-card">
<div class="stat-label">Best Lap</div>
<div class="stat-value">
''' + str(int(best_lap_time // 60)) + ':' + f"{best_lap_time % 60:05.2f}" + '''
<span class="stat-unit">min</span>
</div>
</div>

<div class="stat-card">
<div class="stat-label">Theoretical Best</div>
<div class="stat-value">
''' + str(int(theoretical_best // 60)) + ':' + f"{theoretical_best % 60:05.2f}" + '''
<span class="stat-unit">min</span>
</div>
</div>

<div class="stat-card">
<div class="stat-label">Improvement</div>
<div class="stat-value">
''' + f"{improvement:.3f}" + '''
<span class="stat-unit">sec</span>
</div>
</div>

<div class="stat-card">
<div class="stat-label">Complete Laps</div>
<div class="stat-value">
''' + str(complete_laps_count) + '''
<span class="stat-unit">laps</span>
</div>
</div>

<div class="stat-card">
<div class="stat-label">Avg Speed</div>
<div class="stat-value">
''' + f"{avg_speed:.1f}" + '''
<span class="stat-unit">km/h</span>
</div>
</div>

<div class="stat-card">
<div class="stat-label">Track Length</div>
<div class="stat-value">
''' + f"{track_length:.2f}" + '''
<span class="stat-unit">km</span>
</div>
</div>
</div>
</header>

<div class="container">
<div class="tabs">
<button class="tab active" onclick="switchTab('overview')">Overview</button>
<button class="tab" onclick="switchTab('sectors')">Sector Analysis</button>
<button class="tab" onclick="switchTab('laps')">Lap Times</button>
<button class="tab" onclick="switchTab('track')">Track Map</button>
<button class="tab" onclick="switchTab('viewer')">🎮 Lap Viewer</button>
</div>

<!-- Overview Tab -->
<div id="overview" class="tab-content active">
<div style="display: grid; grid-template-columns: 1fr; gap: 2rem;">
<div style="background: var(--bg-card); border: 1px solid var(--border-color); border-radius: 8px; padding: 1.5rem;">
<h2 style="font-family: 'Orbitron', sans-serif; font-size: 1.25rem; margin-bottom: 1rem;">📊 Track Comparison & Racing Line</h2>
<img src="track_comparison.png" style="width: 100%; border-radius: 4px;">
<p style="margin-top: 1rem; color: var(--text-secondary); line-height: 1.6;">Side-by-side comparison showing the official Barber Motorsports Park layout alongside GPS telemetry data.</p>
</div>
<div style="background: var(--bg-card); border: 1px solid var(--border-color); border-radius: 8px; padding: 1.5rem;">
<h2 style="font-family: 'Orbitron', sans-serif; font-size: 1.25rem; margin-bottom: 1rem;">📈 Performance Analysis Dashboard</h2>
<img src="performance_analysis.png" style="width: 100%; border-radius: 4px;">
<p style="margin-top: 1rem; color: var(--text-secondary); line-height: 1.6;">Comprehensive performance metrics including sector times, lap progression, and consistency analysis.</p>
</div>
</div>
</div>

<!-- Sector Analysis Tab -->
<div id="sectors" class="tab-content">
<div style="background: var(--bg-card); border: 1px solid var(--border-color); border-radius: 8px; padding: 1.5rem;">
<h2 style="font-family: 'Orbitron', sans-serif; font-size: 1.25rem; margin-bottom: 1rem;">⚡ Sector Performance Breakdown</h2>
<table>
<thead style="background: rgba(255, 51, 102, 0.1); border-bottom: 2px solid var(--accent-red);">
<tr>
<th style="font-family: 'Orbitron', sans-serif; color: var(--accent-cyan);">Sector</th>
<th style="font-family: 'Orbitron', sans-serif; color: var(--accent-cyan);">Best Time</th>
<th style="font-family: 'Orbitron', sans-serif; color: var(--accent-cyan);">Best Lap</th>
<th style="font-family: 'Orbitron', sans-serif; color: var(--accent-cyan);">Average</th>
<th style="font-family: 'Orbitron', sans-serif; color: var(--accent-cyan);">Consistency</th>
<th style="font-family: 'Orbitron', sans-serif; color: var(--accent-cyan);">Avg Speed</th>
</tr>
</thead>
<tbody>
'''
    
    for stat in sector_stats:
        html += '<tr>\n'
        html += '<td><span style="font-family: \'Orbitron\', sans-serif; font-weight: 700; font-size: 1.25rem; color: var(--accent-yellow);">' + str(stat['sector']) + '</span></td>\n'
        html += '<td style="color: var(--accent-red); font-weight: 700;">' + f"{stat['best_time']:.3f}" + 's</td>\n'
        html += '<td>Lap ' + str(stat['best_lap']) + '</td>\n'
        html += '<td>' + f"{stat['avg_time']:.3f}" + 's</td>\n'
        html += '<td>±' + f"{stat['std_time']:.3f}" + 's</td>\n'
        html += '<td>' + f"{stat['avg_speed']:.1f}" + ' km/h</td>\n'
        html += '</tr>\n'
    
    html += '''
</tbody>
</table>
<div style="margin-top: 2rem; padding: 1.5rem; background: rgba(255, 51, 102, 0.1); border-left: 4px solid var(--accent-red); border-radius: 4px;">
<h3 style="font-family: 'Orbitron', sans-serif; font-weight: 700; margin-bottom: 1rem; color: var(--accent-red);">🎯 Key Insight</h3>
<p style="color: var(--text-secondary);">Sector ''' + str(most_inconsistent['sector']) + ''' shows the most improvement potential with ±''' + f"{most_inconsistent['std_time']:.3f}" + '''s variance. Focus on consistency to unlock <strong style="color: var(--accent-yellow);">''' + f"{potential_gain:.3f}" + ''' seconds</strong> of potential gains.</p>
</div>
</div>
</div>

<!-- Lap Times Tab -->
<div id="laps" class="tab-content">
<div style="background: var(--bg-card); border: 1px solid var(--border-color); border-radius: 8px; padding: 1.5rem;">
<h2 style="font-family: 'Orbitron', sans-serif; font-size: 1.25rem; margin-bottom: 1rem;">🏆 Top 5 Fastest Laps</h2>
<div style="display: grid; grid-template-columns: repeat(auto-fill, minmax(300px, 1fr)); gap: 1rem; margin-top: 1rem;">
'''
    
    for lap_info in top_laps:
        border_color = 'var(--accent-yellow)' if lap_info['is_best'] else 'var(--accent-cyan)'
        bg_color = 'rgba(255, 215, 0, 0.1)' if lap_info['is_best'] else 'rgba(255, 255, 255, 0.03)'
        
        html += '<div style="background: ' + bg_color + '; border: 1px solid var(--border-color); border-left: 3px solid ' + border_color + '; padding: 1rem; display: flex; justify-content: space-between; align-items: center;">\n'
        html += '<div>\n'
        html += '<div style="font-family: \'Orbitron\', sans-serif; font-weight: 900; font-size: 1.5rem; color: var(--accent-cyan);">#' + str(lap_info['lap']) + '</div>\n'
        html += '<div style="font-size: 0.85rem; color: var(--text-secondary); margin-top: 0.25rem;">S1: ' + f"{lap_info['s1']:.2f}" + ' • S2: ' + f"{lap_info['s2']:.2f}" + ' • S3: ' + f"{lap_info['s3']:.2f}" + '</div>\n'
        html += '</div>\n'
        html += '<div style="font-family: \'Orbitron\', sans-serif; font-weight: 700; font-size: 1.25rem;">' + str(int(lap_info['time'] // 60)) + ':' + f"{lap_info['time'] % 60:05.2f}" + '</div>\n'
        html += '</div>\n'
    
    html += '''
</div>
<div style="margin-top: 2rem; padding: 1.5rem; background: rgba(0, 217, 255, 0.1); border-left: 4px solid var(--accent-cyan); border-radius: 4px;">
<h3 style="font-family: 'Orbitron', sans-serif; font-weight: 700; margin-bottom: 0.5rem; color: var(--accent-cyan);">💡 Performance Summary</h3>
<p style="color: var(--text-secondary); margin-bottom: 1rem;">Average lap time: <strong>''' + str(int(avg_lap_time // 60)) + ':' + f"{avg_lap_time % 60:05.2f}" + '''</strong> with ±''' + f"{lap_times.std():.3f}" + '''s consistency</p>
<p style="color: var(--text-secondary);">Driver is performing at <strong style="color: var(--accent-yellow);">''' + f"{performance_pct:.1f}" + '''%</strong> of theoretical maximum. Only <strong style="color: var(--accent-red);">''' + f"{improvement:.3f}" + ''' seconds</strong> separates the best lap from perfection.</p>
</div>
</div>
</div>

<!-- Track Map Tab -->
<div id="track" class="tab-content">
<div style="background: var(--bg-card); border: 1px solid var(--border-color); border-radius: 8px; padding: 1.5rem;">
<h2 style="font-family: 'Orbitron', sans-serif; font-size: 1.25rem; margin-bottom: 1rem;">🗺️ Track Layout & GPS Coverage</h2>
<div style="display: grid; grid-template-columns: repeat(auto-fit, minmax(300px, 1fr)); gap: 2rem; margin-top: 1rem;">
<div>
<h3 style="font-family: 'Orbitron', sans-serif; font-weight: 700; margin-bottom: 1rem; color: var(--accent-cyan);">Track Sectors</h3>
<ul style="list-style: none; color: var(--text-secondary); line-height: 2;">
<li style="padding: 0.5rem; background: rgba(255, 215, 0, 0.1); border-left: 3px solid var(--accent-yellow); margin-bottom: 0.5rem;"><strong style="color: var(--accent-yellow);">Sector 1:</strong> 0m to ''' + f"{sector_boundaries[1][1]:.0f}" + '''m</li>
<li style="padding: 0.5rem; background: rgba(0, 217, 255, 0.1); border-left: 3px solid var(--accent-cyan); margin-bottom: 0.5rem;"><strong style="color: var(--accent-cyan);">Sector 2:</strong> ''' + f"{sector_boundaries[2][0]:.0f}" + '''m to ''' + f"{sector_boundaries[2][1]:.0f}" + '''m</li>
<li style="padding: 0.5rem; background: rgba(255, 51, 102, 0.1); border-left: 3px solid var(--accent-red);"><strong style="color: var(--accent-red);">Sector 3:</strong> ''' + f"{sector_boundaries[3][0]:.0f}" + '''m to end</li>
</ul>
</div>
<div>
<h3 style="font-family: 'Orbitron', sans-serif; font-weight: 700; margin-bottom: 1rem; color: var(--accent-cyan);">GPS Data Quality</h3>
<ul style="list-style: none; color: var(--text-secondary); line-height: 2;">
<li style="padding: 0.5rem;"><strong>Data Points:</strong> ''' + f"{total_points:,}" + ''' total</li>
<li style="padding: 0.5rem;"><strong>Complete Laps:</strong> ''' + str(complete_laps_count) + ''' laps</li>
<li style="padding: 0.5rem;"><strong>Best Lap:</strong> Lap ''' + str(best_lap_num) + '''</li>
<li style="padding: 0.5rem;"><strong>Avg Lap Time:</strong> ''' + str(int(avg_lap_time // 60)) + ':' + f"{avg_lap_time % 60:05.2f}" + '''</li>
</ul>
</div>
</div>
<div style="margin-top: 2rem;">
<img src="track_comparison.png" style="width: 100%; border-radius: 4px;">
</div>
</div>
</div>

<!-- Lap Viewer Tab -->
<div id="viewer" class="tab-content">
<div id="lap-viewer-root"></div>
</div>
</div>

<script>
function switchTab(tabName) {
  const tabContents = document.querySelectorAll('.tab-content');
  tabContents.forEach(content => content.classList.remove('active'));
  const tabs = document.querySelectorAll('.tab');
  tabs.forEach(tab => tab.classList.remove('active'));
  document.getElementById(tabName).classList.add('active');
  event.target.classList.add('active');
}
</script>

<script type="text/babel">
const { useState, useEffect, useRef } = React;

const LAP_DATA = ''' + lap_data_str + ''';
const BEST_SECTORS = ''' + best_sectors_str + ''';
const VEHICLE_ID = "''' + vehicle_id + '''";

console.log('Data loaded:', Object.keys(LAP_DATA).length, 'laps');

const getSpeedColor = (speed, minSpeed, maxSpeed) => {
    const normalized = Math.max(0, Math.min(1, (speed - minSpeed) / (maxSpeed - minSpeed)));
    if (normalized < 0.25) {
        const t = normalized / 0.25;
        return `rgb(${Math.floor(13 + t * 63)}, ${Math.floor(8 + t * 42)}, ${Math.floor(135 + t * 27)})`;
    } else if (normalized < 0.5) {
        const t = (normalized - 0.25) / 0.25;
        return `rgb(${Math.floor(76 + t * 79)}, ${Math.floor(50 + t * 9)}, ${Math.floor(162 - t * 20)})`;
    } else if (normalized < 0.75) {
        const t = (normalized - 0.5) / 0.25;
        return `rgb(${Math.floor(155 + t * 82)}, ${Math.floor(59 + t * 62)}, ${Math.floor(142 - t * 59)})`;
    } else {
        const t = (normalized - 0.75) / 0.25;
        return `rgb(${Math.floor(237 + t * 3)}, ${Math.floor(121 + t * 102)}, ${Math.floor(83 + t * 7)})`;
    }
};

const SECTOR_COLORS = {
    0: '#ffffff', 1: '#ffd700', 2: '#00d9ff', 3: '#ff3366'
};

const GEAR_COLORS = {
    0: '#666666', 1: '#4a90e2', 2: '#50c878', 3: '#ffd700',
    4: '#ff8c00', 5: '#ff4500', 6: '#ff1493'
};

function LapViewer() {
    const laps = Object.keys(LAP_DATA).map(Number).sort((a, b) => a - b);
    const [currentLapIndex, setCurrentLapIndex] = useState(0);
    const [highlightMode, setHighlightMode] = useState('speed');
    const canvasRef = useRef(null);
    
    const currentLap = laps[currentLapIndex];
    const lapData = LAP_DATA[currentLap];
    
    const allSpeeds = Object.values(LAP_DATA).flatMap(lap => lap.speed.map(s => Number(s)).filter(s => !isNaN(s)));
    const minSpeed = Math.min(...allSpeeds);
    const maxSpeed = Math.max(...allSpeeds);
    
    useEffect(() => {
        const canvas = canvasRef.current;
        if (!canvas || !lapData) return;
        
        const ctx = canvas.getContext('2d');
        const width = canvas.width;
        const height = canvas.height;
        
        ctx.fillStyle = '#13171f';
        ctx.fillRect(0, 0, width, height);
        
        const allX = Object.values(LAP_DATA).flatMap(lap => lap.x);
        const allY = Object.values(LAP_DATA).flatMap(lap => lap.y);
        const minX = Math.min(...allX);
        const maxX = Math.max(...allX);
        const minY = Math.min(...allY);
        const maxY = Math.max(...allY);
        
        const padding = 50;
        const scaleX = (width - 2 * padding) / (maxX - minX);
        const scaleY = (height - 2 * padding) / (maxY - minY);
        const scale = Math.min(scaleX, scaleY);
        
        const offsetX = (width - (maxX - minX) * scale) / 2;
        const offsetY = (height - (maxY - minY) * scale) / 2;
        
        const toCanvasX = (x) => (x - minX) * scale + offsetX;
        const toCanvasY = (y) => height - ((y - minY) * scale + offsetY);
        
        // Draw background laps
        ctx.strokeStyle = 'rgba(255, 255, 255, 0.1)';
        ctx.lineWidth = 15;
        ctx.lineCap = 'round';
        ctx.lineJoin = 'round';
        
        Object.values(LAP_DATA).forEach(lap => {
            ctx.beginPath();
            lap.x.forEach((x, i) => {
                const cx = toCanvasX(x);
                const cy = toCanvasY(lap.y[i]);
                if (i === 0) ctx.moveTo(cx, cy);
                else ctx.lineTo(cx, cy);
            });
            ctx.stroke();
        });
        
        // Draw white base layer for current lap
        ctx.strokeStyle = 'rgba(255, 255, 255, 0.8)';
        ctx.lineWidth = 12;
        ctx.lineCap = 'round';
        ctx.lineJoin = 'round';
        ctx.beginPath();
        lapData.x.forEach((x, i) => {
            const cx = toCanvasX(x);
            const cy = toCanvasY(lapData.y[i]);
            if (i === 0) ctx.moveTo(cx, cy);
            else ctx.lineTo(cx, cy);
        });
        ctx.stroke();
        
        // Draw colored paths grouped by color
        ctx.lineWidth = 10;
        
        let outlierInfo = [];
        const minSpacing = 1.0;
        
        for (let i = 0; i < lapData.x.length; i++) {
            const x = toCanvasX(lapData.x[i]);
            const y = toCanvasY(lapData.y[i]);
            
            let colorBucket;
            if (highlightMode === 'speed') {
                const speed = Number(lapData.speed[i]);
                const speedBucket = Math.round(speed / 5) * 5;
                colorBucket = getSpeedColor(speedBucket, minSpeed, maxSpeed);
            } else if (highlightMode === 'gear') {
                const gear = lapData.gear[i] || 0;
                colorBucket = GEAR_COLORS[gear] || '#ffffff';
            } else {
                const sector = lapData.sector[i] || 0;
                colorBucket = SECTOR_COLORS[sector];
            }
            
            outlierInfo.push({ x, y, color: colorBucket });
        }
        
        // Draw continuous paths
        let currentColor = null;
        ctx.beginPath();
        
        for (let i = 0; i < outlierInfo.length; i++) {
            const point = outlierInfo[i];
            
            if (point.color !== currentColor) {
                if (currentColor !== null) {
                    ctx.stroke();
                }
                ctx.strokeStyle = point.color;
                ctx.beginPath();
                ctx.moveTo(point.x, point.y);
                currentColor = point.color;
            } else {
                ctx.lineTo(point.x, point.y);
            }
        }
        ctx.stroke();
        
        // Draw start/finish marker
        const startX = toCanvasX(lapData.x[0]);
        const startY = toCanvasY(lapData.y[0]);
        ctx.fillStyle = '#00ff00';
        ctx.beginPath();
        ctx.arc(startX, startY, 12, 0, Math.PI * 2);
        ctx.fill();
        
    }, [currentLap, highlightMode]);
    
    useEffect(() => {
        const handleKey = (e) => {
            if (e.key === 'ArrowLeft') setCurrentLapIndex(p => Math.max(0, p - 1));
            else if (e.key === 'ArrowRight') setCurrentLapIndex(p => Math.min(laps.length - 1, p + 1));
            else if (e.key === 's') setHighlightMode('speed');
            else if (e.key === 'g') setHighlightMode('gear');
            else if (e.key === 't') setHighlightMode('sector');
        };
        window.addEventListener('keydown', handleKey);
        return () => window.removeEventListener('keydown', handleKey);
    }, [laps.length]);
    
    const hasBest = Object.entries(BEST_SECTORS).some(([s, i]) => i.lap === currentLap);
    const bestSecs = Object.entries(BEST_SECTORS).filter(([s, i]) => i.lap === currentLap).map(([s]) => s);
    
    return (
        <div id="lap-viewer-container">
            <div className="canvas-container">
                <canvas ref={canvasRef} width={1200} height={900} id="lap-canvas" />
            </div>
            
            <div className="viewer-panel">
                <div>
                    <h1 style={{fontFamily: "'Orbitron', sans-serif", fontSize: '1.5rem', fontWeight: 900, color: '#ff3366'}}>
                        LAP VIEWER
                    </h1>
                    <div style={{fontFamily: "'Orbitron', sans-serif", color: '#ffd700', marginTop: '0.5rem'}}>
                        {VEHICLE_ID}
                    </div>
                </div>
                
                <div style={{background: 'rgba(255,255,255,0.05)', border: hasBest ? '2px solid #ffd700' : '1px solid #2d3748', borderRadius: '8px', padding: '20px'}}>
                    <div style={{fontFamily: "'Orbitron', sans-serif", fontSize: '2.5rem', fontWeight: 900, color: '#00d9ff'}}>
                        LAP {currentLap}
                    </div>
                    
                    {hasBest && (
                        <div style={{background: 'rgba(255,215,0,0.2)', border: '1px solid #ffd700', borderRadius: '4px', padding: '10px', marginTop: '10px', color: '#ffd700'}}>
                            ⭐ BEST SECTOR{bestSecs.length > 1 ? 'S' : ''}: {bestSecs.join(', ')}
                        </div>
                    )}
                    
                    <div style={{fontSize: '0.85rem', color: '#9ca3af', marginTop: '15px'}}>TOTAL TIME</div>
                    <div style={{fontFamily: "'Orbitron', sans-serif", fontSize: '1.5rem', fontWeight: 700, color: '#e8e8e8'}}>
                        {Math.floor(lapData.total_time / 60)}:{(lapData.total_time % 60).toFixed(3).padStart(6, '0')}
                    </div>
                    
                    <div style={{marginTop: '15px', display: 'flex', flexDirection: 'column', gap: '8px'}}>
                        {Object.entries(lapData.sector_times).map(([s, t]) => {
                            const isBest = BEST_SECTORS[s]?.lap === currentLap;
                            return (
                                <div key={s} style={{display: 'flex', justifyContent: 'space-between', padding: '8px', background: isBest ? 'rgba(255,215,0,0.1)' : 'rgba(255,255,255,0.03)', borderLeft: `3px solid ${SECTOR_COLORS[s]}`, borderRadius: '4px'}}>
                                    <span>Sector {s}</span>
                                    <span style={{fontFamily: "'Orbitron', sans-serif", color: isBest ? '#ffd700' : '#e8e8e8'}}>
                                        {t.toFixed(3)}s {isBest && '🏆'}
                                    </span>
                                </div>
                            );
                        })}
                    </div>
                </div>
                
                <div>
                    <div style={{fontSize: '0.85rem', color: '#9ca3af', marginBottom: '10px', textTransform: 'uppercase'}}>
                        DISPLAY MODE
                    </div>
                    <div style={{display: 'flex', gap: '10px', flexWrap: 'wrap'}}>
                        <button onClick={() => setHighlightMode('speed')} className={'mode-button' + (highlightMode === 'speed' ? ' active' : '')}>
                            Speed
                        </button>
                        <button onClick={() => setHighlightMode('gear')} className={'mode-button' + (highlightMode === 'gear' ? ' active' : '')}>
                            Gear
                        </button>
                        <button onClick={() => setHighlightMode('sector')} className={'mode-button' + (highlightMode === 'sector' ? ' active' : '')}>
                            Sectors
                        </button>
                    </div>
                    
                    {highlightMode === 'gear' && (
                        <div style={{padding: '15px', background: 'rgba(255,255,255,0.05)', borderRadius: '4px', marginTop: '15px'}}>
                            <div style={{fontWeight: 600, marginBottom: '10px', color: '#e8e8e8', fontSize: '0.9rem'}}>Gear Legend:</div>
                            <div style={{display: 'grid', gridTemplateColumns: '1fr 1fr', gap: '8px', fontSize: '0.85rem'}}>
                                {[1, 2, 3, 4, 5, 6].map(g => (
                                    <div key={g} style={{display: 'flex', alignItems: 'center', gap: '8px'}}>
                                        <div style={{width: '20px', height: '20px', background: GEAR_COLORS[g], borderRadius: '3px', border: '1px solid rgba(255,255,255,0.2)'}}></div>
                                        <span>Gear {g}</span>
                                    </div>
                                ))}
                            </div>
                        </div>
                    )}
                    
                    {highlightMode === 'speed' && (
                        <div style={{padding: '15px', background: 'rgba(255,255,255,0.05)', borderRadius: '4px', marginTop: '15px'}}>
                            <div style={{fontWeight: 600, marginBottom: '10px', color: '#e8e8e8', fontSize: '0.9rem'}}>Speed Range:</div>
                            <div style={{display: 'flex', alignItems: 'center', gap: '10px', fontSize: '0.85rem'}}>
                                <div style={{flex: 1, height: '20px', background: 'linear-gradient(90deg, rgb(13,8,135), rgb(76,50,162), rgb(155,59,142), rgb(237,121,83), rgb(240,223,90))', borderRadius: '3px'}}></div>
                            </div>
                            <div style={{display: 'flex', justifyContent: 'space-between', marginTop: '5px', fontSize: '0.75rem', color: '#9ca3af'}}>
                                <span>{Math.round(minSpeed)} km/h</span>
                                <span>{Math.round(maxSpeed)} km/h</span>
                            </div>
                        </div>
                    )}
                </div>
                
                <div style={{display: 'flex', gap: '10px'}}>
                    <button onClick={() => setCurrentLapIndex(Math.max(0, currentLapIndex - 1))} disabled={currentLapIndex === 0} className="nav-button">
                        ← PREV
                    </button>
                    <button onClick={() => setCurrentLapIndex(Math.min(laps.length - 1, currentLapIndex + 1))} disabled={currentLapIndex === laps.length - 1} className="nav-button">
                        NEXT →
                    </button>
                </div>
                
                <div>
                    <div style={{fontSize: '0.85rem', color: '#9ca3af', marginBottom: '10px', textAlign: 'center'}}>
                        LAP {currentLapIndex + 1} OF {laps.length}
                    </div>
                    <div style={{width: '100%', height: '4px', background: 'rgba(255,255,255,0.1)', borderRadius: '2px', overflow: 'hidden'}}>
                        <div style={{width: `${((currentLapIndex + 1) / laps.length) * 100}%`, height: '100%', background: 'linear-gradient(90deg, #ff3366, #00d9ff)', transition: 'width 0.3s'}} />
                    </div>
                </div>
                
                <div style={{padding: '15px', background: 'rgba(255,255,255,0.03)', borderRadius: '4px', fontSize: '0.85rem', color: '#9ca3af'}}>
                    <div style={{fontWeight: 600, marginBottom: '8px', color: '#e8e8e8'}}>Keyboard Shortcuts:</div>
                    <div>← → Navigate laps</div>
                    <div>S - Speed mode</div>
                    <div>G - Gear mode</div>
                    <div>T - Sector mode</div>
                </div>
            </div>
        </div>
    );
}

ReactDOM.render(<LapViewer />, document.getElementById('lap-viewer-root'));
</script>

</body>
</html>
'''
    
    return html


# ==================== MAIN EXECUTION ====================
def main():
    """Main execution function"""
    
    # Load data
    df_track = load_telemetry_data(VEHICLE_ID)
    
    # Transform coordinates
    df_track, markers = transform_coordinates(df_track)
    
    # Convert to pandas for processing
    df_track_pd = pd.DataFrame(df_track.to_dict(as_series=False))
    
    # Clean GPS data
    df_track_clean = clean_gps_data(df_track_pd, MAX_GAP_THRESHOLD, MIN_POINTS_PER_LAP)
    
    # Get reference lap
    lap_counts = df_track_clean.groupby('lap').size().sort_values(ascending=False)
    best_lap = lap_counts.index[0]
    df_reference_lap = df_track_clean[df_track_clean['lap'] == best_lap].copy()
    
    print(f"\n   Using lap {best_lap} as reference ({len(df_reference_lap)} points)")
    
    # Assign sectors
    df_with_sectors = assign_sectors(df_track_clean, df_reference_lap)
    
    # Calculate ideal racing line
    ideal_line = calculate_ideal_line(df_with_sectors)
    
    # Calculate sector times
    sector_analysis = calculate_sector_times(df_with_sectors, MIN_SPEED_THRESHOLD)
    
    # Analyze complete laps
    complete_sectors, lap_times = analyze_complete_laps(sector_analysis, VEHICLE_ID)
    
    # Create visualizations
    print(f"\n📊 Generating visualizations...")
    
    fig1 = plot_track_comparison(
        df_track_clean, 
        df_reference_lap,
        ideal_line,
        markers,
        metric_column='Speed_kph',
        image_path=TRACK_IMAGE,
        max_gap=MAX_GAP_THRESHOLD
    )
    plt.savefig(f'{OUTPUT_DIR}/track_comparison.png', dpi=300, bbox_inches='tight')
    print(f"   ✓ Saved track_comparison.png")
    plt.close()
    
    fig2 = plot_performance_analysis(complete_sectors, lap_times, VEHICLE_ID)
    plt.savefig(f'{OUTPUT_DIR}/performance_analysis.png', dpi=300, bbox_inches='tight')
    print(f"   ✓ Saved performance_analysis.png")
    plt.close()
    
    # Generate interactive lap viewer
    # Generate unified HTML dashboard with lap viewer
    print(f"\n🌐 Generating unified dashboard with lap viewer...")
    unified_html = generate_unified_dashboard(complete_sectors, lap_times, VEHICLE_ID, df_with_sectors)
    
    unified_path = f'{OUTPUT_DIR}/telemetry_dashboard_{VEHICLE_ID.replace("-", "_")}.html'
    with open(unified_path, 'w', encoding='utf-8') as f:
        f.write(unified_html)
    
    print(f"   ✓ Saved: {unified_path}")
    
    print("\n✅ Analysis complete!")
    print(f"\n📁 Generated Files:")
    print(f"   🌐 Unified Dashboard: {unified_path}")
    print(f"   📊 Track Comparison: track_comparison.png")
    print(f"   📊 Performance Analysis: performance_analysis.png")
    print(f"\n💡 Open {unified_path} in your browser and click the '🎮 Lap Viewer' tab!")
    
    return df_with_sectors, complete_sectors, lap_times, ideal_line

# Run the analysis
if __name__ == "__main__":
    df_with_sectors, complete_sectors, lap_times, ideal_line = main()

MOTORSPORTS TELEMETRY ANALYSIS SYSTEM
Barber Motorsports Park

📥 Loading data for GR86-002-000...
   Loaded 586,236 raw telemetry rows
   Processed 59,811 telemetry points

🗺️  Converting coordinates...
   Using 92793.50 meters per degree longitude

🧹 Cleaning GPS data...
   Found gaps in 25 laps
   Keeping 28 out of 28 laps

   Using lap 4 as reference (3031 points)

📍 Assigning sectors...
   Sector 1: 0m to 1029m
   Sector 2: 1029m to 2609m
   Sector 3: 2609m to end

🏁 Calculating ideal racing line...
   Ideal line uses 712 segments from 28 laps
   Average speed: 143.2 km/h

⏱️  Calculating sector times...
   Calculated sector times for 80 sector-lap combinations

   Complete laps: 26 out of 28

🏁 PERFORMANCE SUMMARY - GR86-002-000

📊 LAP TIMES:
   Best Lap: Lap 20 - 94.097s (1.57 min)
   Average: 98.677s (1.64 min)
   Consistency: ±7.933s

⚡ THEORETICAL BEST LAP: 91.206s (1.52 min)
   Sector 1: 26.035s (Lap 13)
   Sector 2: 39.563s (Lap 20)
   Sector 3: 25.607s (Lap 26)

💡 Improveme